# HW6

Amirhossein hassni - 99103803

Importing Libraries:

In [2]:
import pandas as pd
import numpy as np
from math import exp, log
import matplotlib.pyplot as plt

cost_matrix represents the cost of transportation between origins and destinations. The value X represents infinite cost, indicating infeasible connections.

In [3]:
X = float('inf')

cost_matrix = np.array([
    [X, X, 50, X, X],
    [X, X, 60, X, X],
    [X, X, X, 30, X],
    [20, X, 80, X, 20],
    [X, 70, 90, 10, X]
   
])

cost_matrix = cost_matrix.astype(float)

origins and destinations vectors representing the supply (origins) and demand (destinations) in the future.

In [5]:
origins = np.array([80, 150, 140, 160, 180])
destinations = np.array([40, 90, 320, 80, 200])

In [6]:
sum_destinations = np.sum(destinations)
sum_origins = np.sum(origins)

if sum_origins != sum_destinations:
    scaling_factor = sum_origins / sum_destinations
    destinations = destinations * scaling_factor

In [7]:
destinations

array([ 38.90410959,  87.53424658, 311.23287671,  77.80821918,
       194.52054795])

Adjusts the destination vector to make the total supply equal to the total demand.

In [10]:
def cost(c_ij):
    return 0.05 * exp(-0.2 * (log(c_ij + 1))**2)

estimated_costs = np.zeros_like(cost_matrix, dtype=float)

for i in range(cost_matrix.shape[0]):
    for j in range(cost_matrix.shape[1]):
        if i != j:
            estimated_costs[i, j] = cost(cost_matrix[i, j])

Defines a cost function and estimates transportation costs using this function.

In [11]:
estimated_costs

array([[0.        , 0.        , 0.00227089, 0.        , 0.        ],
       [0.        , 0.        , 0.00170262, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00472832, 0.        ],
       [0.00783186, 0.        , 0.00105105, 0.        , 0.00783186],
       [0.        , 0.0013204 , 0.00085424, 0.01583215, 0.        ]])

In [19]:
def gravity(cost_matrix, origins, destinations):
    error = float('inf')
    n = len(origins)
    factors_origins = np.ones(n)
    factors_destinations = np.ones(n)
    total_attractiveness = np.sum(origins)
    OD = np.zeros_like(cost_matrix, dtype=float)
    iterations = 0
    error_threshold = 0.01
    while error > error_threshold:
        for i in range(n):
            factors_origins[i] = 1 / (np.sum(factors_destinations * destinations * estimated_costs[i, :]) + 1e-8)

        for j in range(n):
            factors_destinations[j] = 1 / (np.sum(factors_origins * origins * estimated_costs[:, j]) + 1e-8)

        OD = np.outer(factors_origins * origins, factors_destinations * destinations) * estimated_costs

        
        error = (np.sum(np.abs(origins - np.sum(OD, axis=1))) +
                 np.sum(np.abs(destinations - np.sum(OD, axis=0)))) / total_attractiveness
        
        iterations += 1
        if iterations>3000:
            break
        
    OD = pd.DataFrame(OD)
    print(f"Error: {error * 100:.3f}%")
    return OD


Implements the gravity algorithm to find the optimal distribution of goods between origins and destinations.

The algorithm iteratively adjusts the attractiveness factors for origins and destinations until the error falls below a threshold.

In [20]:
result_OD = gravity(cost_matrix, origins, destinations)

Error: 20.683%


Calls the gravity function and prints the error.

In [21]:
result_OD

,0,1,2,3,4
0,0.000000,0.000000,7.929653e+01,0.000000,0.000000
1,0.000000,0.000000,1.482459e+02,0.000000,0.000000
2,0.000000,0.000000,0.000000e+00,71.006629,0.000000
3,26.666667,0.000000,1.451602e-12,0.000000,133.333333
4,0.000000,87.534247,8.369044e+01,6.801590,0.000000


The final OD matrix 

# Conclusion

 
The code provides a practical implementation of the gravity model, a mathematical model commonly used in logistics and transportation planning. It aims to optimize the flow of trasportation between multiple origins and destinations based on transportation costs and supply-demand constraints. The algorithm iteratively refines the distribution until a satisfactory solution is reached.

# Q2

In instances where there is no movement between two zones, it becomes crucial to designate the travel cost between these zones with a value of either infinity or an exceedingly large figure that effectively approaches infinity. This strategic decision underscores the absence of a direct link or feasible travel route between the specified zones. The allocation of an impractically high cost to travel between these zones serves to convey their lack of practical connectivity within the framework of the employed model, whether it be the Gravity Model. This approach is not limited to the Gravity Model; similar principles are applied in various network analysis and optimization models. It's a way of enforcing the logical constraints of the system being modeled and ensuring that the results align with practical considerations.